In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install required packages
!pip install wandb torch torchvision pandas numpy matplotlib seaborn scikit-learn

# Set up Kaggle API
!pip install kaggle

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 58.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 92.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
# Upload your kaggle.json to Colab and run:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/ColabNotebooks/kaggle_API_credentials/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [4]:
# Download the dataset
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting
!unzip -q walmart-recruiting-store-sales-forecasting.zip

  0% 0.00/2.70M [00:00<?, ?B/s]
100% 2.70M/2.70M [00:00<00:00, 705MB/s]


In [5]:
!unzip -q train.csv.zip
!unzip -q stores.csv.zip
!unzip -q test.csv.zip
!unzip -q features.csv.zip

unzip:  cannot find or open stores.csv.zip, stores.csv.zip.zip or stores.csv.zip.ZIP.


In [6]:
!pip install mlflow


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.7/24.7 MB 72.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.9/114.9 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 733.8/733.8 kB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.4/203.4 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 15.5 MB/s eta 0:00:00


In [8]:
# model_experiment_nbeats.ipynb

import torch
import torch.nn as nn
import numpy as np
import pandas as pd
from torch.utils.data import Dataset, DataLoader
import mlflow
import mlflow.pytorch
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt

class NBEATSBlock(nn.Module):
    def __init__(self, input_size, theta_size, basis_function, layers, layer_size):
        super().__init__()
        self.layers = nn.ModuleList([nn.Linear(input_size, layer_size)] +
                                    [nn.Linear(layer_size, layer_size) for _ in range(layers-1)])
        self.basis_function = basis_function
        self.backcast_fc = nn.Linear(layer_size, theta_size)
        self.forecast_fc = nn.Linear(layer_size, theta_size)

    def forward(self, x):
        # Forward through fully connected layers
        for layer in self.layers:
            x = torch.relu(layer(x))

        # Generate theta parameters
        backcast_theta = self.backcast_fc(x)
        forecast_theta = self.forecast_fc(x)

        # Apply basis functions
        backcast, forecast = self.basis_function(backcast_theta, forecast_theta)

        return backcast, forecast

class GenericBasis(nn.Module):
    def __init__(self, backcast_size, forecast_size):
        super().__init__()
        self.backcast_size = backcast_size
        self.forecast_size = forecast_size

    def forward(self, backcast_theta, forecast_theta):
        # Generic basis function
        backcast = backcast_theta[:, :self.backcast_size]
        forecast = forecast_theta[:, :self.forecast_size]
        return backcast, forecast

class SeasonalityBasis(nn.Module):
    def __init__(self, harmonics, backcast_size, forecast_size):
        super().__init__()
        self.harmonics = harmonics
        self.backcast_size = backcast_size
        self.forecast_size = forecast_size

    def forward(self, backcast_theta, forecast_theta):
        # Seasonality basis using Fourier series
        backcast_harmonics = []
        forecast_harmonics = []

        for i in range(self.harmonics):
            cos_coef = backcast_theta[:, i].unsqueeze(-1)
            sin_coef = backcast_theta[:, i + self.harmonics].unsqueeze(-1)

            # Create time indices
            backcast_time = torch.linspace(0, 1, self.backcast_size).unsqueeze(0)
            forecast_time = torch.linspace(0, 1, self.forecast_size).unsqueeze(0)

            # Apply Fourier basis
            backcast_harmonics.append(
                cos_coef * torch.cos(2 * np.pi * (i + 1) * backcast_time) +
                sin_coef * torch.sin(2 * np.pi * (i + 1) * backcast_time)
            )

            cos_coef_f = forecast_theta[:, i].unsqueeze(-1)
            sin_coef_f = forecast_theta[:, i + self.harmonics].unsqueeze(-1)

            forecast_harmonics.append(
                cos_coef_f * torch.cos(2 * np.pi * (i + 1) * forecast_time) +
                sin_coef_f * torch.sin(2 * np.pi * (i + 1) * forecast_time)
            )

        backcast = torch.sum(torch.stack(backcast_harmonics), dim=0)
        forecast = torch.sum(torch.stack(forecast_harmonics), dim=0)

        return backcast, forecast

class TrendBasis(nn.Module):
    def __init__(self, degree_of_polynomial, backcast_size, forecast_size):
        super().__init__()
        self.degree = degree_of_polynomial
        self.backcast_size = backcast_size
        self.forecast_size = forecast_size

    def forward(self, backcast_theta, forecast_theta):
        # Polynomial trend basis
        backcast_time = torch.linspace(-1, 1, self.backcast_size).unsqueeze(0)
        forecast_time = torch.linspace(-1, 1, self.forecast_size).unsqueeze(0)

        backcast = torch.zeros_like(backcast_time)
        forecast = torch.zeros_like(forecast_time)

        for i in range(self.degree + 1):
            backcast += backcast_theta[:, i].unsqueeze(-1) * (backcast_time ** i)
            forecast += forecast_theta[:, i].unsqueeze(-1) * (forecast_time ** i)

        return backcast, forecast

class NBEATS(nn.Module):
    def __init__(self, backcast_length, forecast_length,
                 stack_types=['generic', 'seasonality', 'trend'],
                 nb_blocks_per_stack=3, hidden_layer_units=128,
                 nb_harmonics=10, polynomial_degree=3):
        super().__init__()
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length
        self.hidden_layer_units = hidden_layer_units
        self.nb_blocks_per_stack = nb_blocks_per_stack
        self.stack_types = stack_types

        self.stacks = nn.ModuleList()

        for stack_type in stack_types:
            stack_blocks = nn.ModuleList()

            for _ in range(nb_blocks_per_stack):
                if stack_type == 'generic':
                    theta_size = backcast_length + forecast_length
                    basis_function = GenericBasis(backcast_length, forecast_length)
                elif stack_type == 'seasonality':
                    theta_size = 2 * nb_harmonics
                    basis_function = SeasonalityBasis(nb_harmonics, backcast_length, forecast_length)
                elif stack_type == 'trend':
                    theta_size = polynomial_degree + 1
                    basis_function = TrendBasis(polynomial_degree, backcast_length, forecast_length)

                block = NBEATSBlock(
                    input_size=backcast_length,
                    theta_size=theta_size,
                    basis_function=basis_function,
                    layers=4,
                    layer_size=hidden_layer_units
                )
                stack_blocks.append(block)

            self.stacks.append(stack_blocks)

    def forward(self, backcast):
        forecast = torch.zeros(backcast.size(0), self.forecast_length)

        for stack in self.stacks:
            for block in stack:
                b, f = block(backcast)
                backcast = backcast - b
                forecast = forecast + f

        return forecast

class WalmartDataset(Dataset):
    def __init__(self, data, backcast_length, forecast_length):
        self.data = data
        self.backcast_length = backcast_length
        self.forecast_length = forecast_length

    def __len__(self):
        return len(self.data) - self.backcast_length - self.forecast_length + 1

    def __getitem__(self, idx):
        backcast = self.data[idx:idx + self.backcast_length]
        forecast = self.data[idx + self.backcast_length:idx + self.backcast_length + self.forecast_length]
        return torch.FloatTensor(backcast), torch.FloatTensor(forecast)

def train_nbeats(model, train_loader, val_loader, epochs=100, lr=0.001):
    """N-BEATS მოდელის ტრენინგი"""

    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        # Training
        model.train()
        train_loss = 0.0

        for backcast, forecast in train_loader:
            optimizer.zero_grad()
            predicted = model(backcast)
            loss = criterion(predicted, forecast)
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0.0

        with torch.no_grad():
            for backcast, forecast in val_loader:
                predicted = model(backcast)
                loss = criterion(predicted, forecast)
                val_loss += loss.item()

        train_losses.append(train_loss / len(train_loader))
        val_losses.append(val_loss / len(val_loader))

        if epoch % 10 == 0:
            print(f'Epoch {epoch}: Train Loss: {train_losses[-1]:.6f}, Val Loss: {val_losses[-1]:.6f}')

    return train_losses, val_losses

# ძირითადი ფუნქცია N-BEATS ექსპერიმენტისთვის
def run_nbeats_experiment():
    """N-BEATS ექსპერიმენტის გაშვება"""

    # დატის ჩატვირთვა
    df = pd.read_csv('train.csv')
    df['Date'] = pd.to_datetime(df['Date'])

    # MLflow ექსპერიმენტის დაწყება
    mlflow.set_experiment("Walmart_NBEATS_Forecasting")

    with mlflow.start_run():
        # პარამეტრები
        BACKCAST_LENGTH = 52  # 52 კვირა
        FORECAST_LENGTH = 4   # 4 კვირის პროგნოზი
        BATCH_SIZE = 32
        EPOCHS = 200
        LEARNING_RATE = 0.001

        # პარამეტრების ლოგირება
        mlflow.log_param("backcast_length", BACKCAST_LENGTH)
        mlflow.log_param("forecast_length", FORECAST_LENGTH)
        mlflow.log_param("batch_size", BATCH_SIZE)
        mlflow.log_param("epochs", EPOCHS)
        mlflow.log_param("learning_rate", LEARNING_RATE)

        # დატასეტის გამზადება
        weekly_sales = df.groupby('Date')['Weekly_Sales'].sum().values

        # Train/Val/Test split
        train_size = int(0.7 * len(weekly_sales))
        val_size = int(0.15 * len(weekly_sales))

        train_data = weekly_sales[:train_size]
        val_data = weekly_sales[train_size:train_size + val_size]
        test_data = weekly_sales[train_size + val_size:]

        # DataLoaders
        train_dataset = WalmartDataset(train_data, BACKCAST_LENGTH, FORECAST_LENGTH)
        val_dataset = WalmartDataset(val_data, BACKCAST_LENGTH, FORECAST_LENGTH)

        train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

        # მოდელის ინიციალიზაცია
        model = NBEATS(
            backcast_length=BACKCAST_LENGTH,
            forecast_length=FORECAST_LENGTH,
            stack_types=['trend', 'seasonality', 'generic'],
            nb_blocks_per_stack=3,
            hidden_layer_units=128
        )

        # ტრენინგი
        train_losses, val_losses = train_nbeats(model, train_loader, val_loader, EPOCHS, LEARNING_RATE)

        # მეტრიკების ლოგირება
        mlflow.log_metric("final_train_loss", train_losses[-1])
        mlflow.log_metric("final_val_loss", val_losses[-1])

        # მოდელის შენახვა
        mlflow.pytorch.log_model(model, "nbeats_model")

        # ტესტირება
        test_dataset = WalmartDataset(test_data, BACKCAST_LENGTH, FORECAST_LENGTH)
        test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

        model.eval()
        predictions = []
        actuals = []

        with torch.no_grad():
            for backcast, forecast in test_loader:
                pred = model(backcast)
                predictions.extend(pred.numpy().flatten())
                actuals.extend(forecast.numpy().flatten())

        # ტესტის მეტრიკები
        mae = mean_absolute_error(actuals, predictions)
        mse = mean_squared_error(actuals, predictions)
        rmse = np.sqrt(mse)

        mlflow.log_metric("test_mae", mae)
        mlflow.log_metric("test_mse", mse)
        mlflow.log_metric("test_rmse", rmse)

        print(f"Test MAE: {mae:.2f}")
        print(f"Test RMSE: {rmse:.2f}")

        return model, predictions, actuals

if __name__ == "__main__":
    model, predictions, actuals = run_nbeats_experiment()

2025/06/27 08:28:39 INFO mlflow.tracking.fluent: Experiment with name 'Walmart_NBEATS_Forecasting' does not exist. Creating a new experiment.


RuntimeError: output with shape [1, 52] doesn't match the broadcast shape [32, 52]